`importing tools`
---

In [2]:
import cv2
import numpy as np
from itertools import combinations
import itertools
import math

`def`
---

[김민아]
- preprocess_for_edge_detection
- detect_edges_and_close
- get_largest_quad_contour
- color_combined
- detect_lines_outlier, draw_lines, intersections_outlier
- sort_corners_consistently, compute_auto_warp_size, warp_perspective

[정아림]
- detect_lines
- intersections
- distance, vector, dot
- is_parallel, polygon_area, is_valid_rectangle, find_largest_rectangle

In [3]:
def preprocess_for_edge_detection(img):
    # 1. 그레이스케일 변환(x) (시각화를 위해 변환하지 않음, 이후 edge 추출 함수에서 변환)  
    gray = img

    # 2. 명도 기반 필터링 (너무 어두운 부분 제거)
    _, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_TOZERO)

    # 3. GaussianBlur로 노이즈 제거 (작은 디테일 삭제)
    blurred = cv2.GaussianBlur(thresh, (7, 7), 0)

    # 4. Bilateral Filter로 경계 유지하며 흐림 적용
    bilateral = cv2.bilateralFilter(blurred, d=9, sigmaColor=75, sigmaSpace=75)

    # 5. Edge Preserving Filter 적용 (명함 경계선 보존)
    edge_preserved = cv2.edgePreservingFilter(bilateral, flags=1, sigma_s=60, sigma_r=0.4)

    return edge_preserved

def detect_edges_and_close(edge_input):
    # 그레이스케일로 변환 후 Canny 적용
    gray = cv2.cvtColor(edge_input, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 75, 200)

    # Morphology 연산 (close) 
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    return closed

def get_largest_quad_contour(edge_img, img_shape):
    # 윤곽선 추출(외부 윤곽선만 + 불필요한 점 생)
    contours, _ = cv2.findContours(edge_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    img_h, img_w = img_shape[:2]
    min_area = img_h * img_w * 0.1  # 이미지 전체 면적의 10% 미만은 건너뛰기 

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < min_area:
            continue

        epsilon = 0.02 * cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True) # 윤곽선 단순화 => corner를 줄이기 

        if len(approx) == 4:
            return approx.reshape(4, 2) # 최종 사각형을 찾은 경우 

    return None # 못 찾은 경우 

def color_combined(img, closed_edges_main):
    # 1. 색상 기반 필터 (밝은 영역만 남기기)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower = np.array([0, 100, 120])
    upper = np.array([180, 255, 255])
    mask = cv2.inRange(hsv, lower, upper) # 범위 내 픽셀만 흰색(255), 그외는 검은색(0)인 mask 생성 

    # 2. 에지 결합 (색상 기반 에지 + 메인 엣지)
    color_edges = cv2.Canny(mask, 50, 150)
    combined_edges = cv2.bitwise_or(closed_edges_main, color_edges)

    return combined_edges

def detect_lines(edge_image, threshold=45, rho_threshold=80, theta_threshold=np.deg2rad(20)):
    """
        using the Hough Line Transform을 사용해 line을 탐색하고 임계값에 따른 유사 lines를 제거합니다.
        - edge_image : input img (edge img)
        - threshold : Hough accumulator에서 선으로 판단할 최소 vote 수
        - rho_threshold : 같은 선으로 판단하는 최소 거리 간격
        - theta_threshold : 같은 선으로 판단하는 최소 각도 간격

    """
    height, width = edge_image.shape
    diag_len = int(np.ceil(np.sqrt(width**2 + height**2))) # 이미지의 대각선 길이
    thetas = np.deg2rad(np.arange(-90, 90)) # 가능한 각도 값들
    rhos = np.arange(-diag_len, diag_len) # 가능한 rho 값들
    accumulator = np.zeros((len(rhos), len(thetas)), dtype=np.uint64) # accumulator 초기화 

    y_idxs, x_idxs = np.nonzero(edge_image)

    for i in range(len(x_idxs)):
        x = x_idxs[i]
        y = y_idxs[i]
        for t_idx in range(len(thetas)):
            rho = int(round(x * np.cos(thetas[t_idx]) + y * np.sin(thetas[t_idx]))) + diag_len
            accumulator[rho, t_idx] += 1

    lines = []
    for r_idx in range(accumulator.shape[0]):
        for t_idx in range(accumulator.shape[1]):
            if accumulator[r_idx, t_idx] >= threshold:
                rho = rhos[r_idx]
                theta = thetas[t_idx]
                lines.append((rho, theta))

    # 유사한 선 제거 → 최종적으로 filtered_lines 반환 
    filtered_lines = []
    for rho, theta in lines:
        is_similar = False
        for frho, ftheta in filtered_lines:
            if abs(rho - frho) < rho_threshold and abs(theta - ftheta) < theta_threshold:
                is_similar = True
                break
        if not is_similar:
            filtered_lines.append((rho, theta))

    return filtered_lines

def intersections(lines, image):
    '''
        lines에 대해서 교차하는 지점만 남깁니다. 이때 근접한 점은 제거하여 최종 filtered_corners를 반환합니다.
        - lines : input
        - image : input image
    '''
    height, width = image.shape[:2]
    intersections = []
    for (rho1, theta1), (rho2, theta2) in combinations(lines, 2):
        A = np.array([
            [np.cos(theta1), np.sin(theta1)],
            [np.cos(theta2), np.sin(theta2)]
        ])
        b = np.array([rho1, rho2])

        if np.linalg.det(A) != 0:
            x0, y0 = np.linalg.solve(A, b)
            x0 = int(round(x0))
            y0 = int(round(y0))
            if 0 <= x0 < width and 0 <= y0 < height:
                intersections.append((x0, y0))

    # 가까운 점 제거(근접한 점 병합)
    filtered_corners = []
    min_dist = 50  # 픽셀 거리 기준

    for pt in intersections:
        is_far = True
        for existing in filtered_corners:
            dist = np.linalg.norm(np.array(pt) - np.array(existing))
            if dist < min_dist:
                is_far = False
                break
        if is_far:
            filtered_corners.append(pt)

    return filtered_corners

def distance(p1, p2):
    '''
        두 점 사이의 거리를 계산하는 함수입니다.
        - p1 : point1
        - p2 : point2
    '''
    return math.hypot(p1[0] - p2[0], p1[1] - p2[1])

def vector(p1, p2):  
    '''
        두 점 p1과 p2로부터 방향 벡터를 계산합니다. (p1에서 p2로 향하는 벡터)
        - p1 : point1
        - p2 : point2
    '''
    return (p2[0] - p1[0], p2[1] - p1[1])

def dot(v1, v2):
    '''
        두 벡터 v1과 v2의 내적(dot product)을 계산합니다.
        - v1 : vector1
        - v2 : vector2
    '''
    return v1[0]*v2[0] + v1[1]*v2[1]

def is_parallel(v1, v2, angle_threshold=10):
    """
        두 벡터가 평행한 지에 대해 확인합니다.
        이때 벡터 간 각도가 angle_threshold 이하거나 180도에 가까우면 평행한 것으로 간주합니다.

        - v1 : vector1
        - v2 : vector2
        - angle_threshold : 허용 각도 오차
    """
    mag1 = math.hypot(*v1)
    mag2 = math.hypot(*v2)
    if mag1 == 0 or mag2 == 0:
        return False
    cos_theta = dot(v1, v2) / (mag1 * mag2)
    angle = math.acos(min(1, max(-1, cos_theta))) * 180 / math.pi
    return angle < angle_threshold or abs(angle - 180) < angle_threshold

def polygon_area(pts):
    """
        신발끈 공식(shoelace formula)을 사용해 주어진 4개의 점으로 이루어진 사각형의 넓이를 계산합니다.
        - pts : 사각형을 이루는 4개의 점 
    """
    x1, y1 = pts[0]
    x2, y2 = pts[1]
    x3, y3 = pts[2]
    x4, y4 = pts[3]
    return 0.5 * abs(
        x1*y2 + x2*y3 + x3*y4 + x4*y1 -
        y1*x2 - y2*x3 - y3*x4 - y4*x1
    )

def is_valid_rectangle(quad, length_threshold=5, angle_threshold=10):
    """
        주어진 4개의 점(quad)이 유효한 직사각형을 형성하는지 확인합니다.
        이때 각도 및 길이 유사성, 평행 여부를 통해 판별합니다.
        - quad : 4개의 꼭짓점 좌표
        - length_threshold : 마주보는 변의 길이 차이 허용 범위
        - angle_threshold : 변 간 평행성 판단을 위한 허용 각도 오차
    """
    quad = sorted(quad, key=lambda p: (p[0], p[1]))  # 일관된 순서를 위해 x, y 기준 정렬
    p1, p2, p3, p4 = quad

     # 다양한 순서 조합으로 변을 구성하여 직사각형 여부 확인
    candidates = [
        [p1, p2, p3, p4],
        [p1, p3, p2, p4],
        [p1, p2, p4, p3],
    ]

    for pts in candidates:
        v1 = vector(pts[0], pts[1])
        v2 = vector(pts[1], pts[2])
        v3 = vector(pts[2], pts[3])
        v4 = vector(pts[3], pts[0])

        len1 = distance(pts[0], pts[1])
        len3 = distance(pts[2], pts[3])
        len2 = distance(pts[1], pts[2])
        len4 = distance(pts[3], pts[0])

        # 마주보는 변들의 평행성과 길이 유사성 검사
        if (is_parallel(v1, v3, angle_threshold) and
            is_parallel(v2, v4, angle_threshold) and
            abs(len1 - len3) <= length_threshold and
            abs(len2 - len4) <= length_threshold):
            return True, pts
    return False, None

def find_largest_rectangle(points, length_threshold=5, angle_threshold=10):
    '''
        가장 넓은 사각형을 만드는 4개의 점을 반환합니다.
        - points : 후보 점들의 리스트
        - length_threshold : 마주보는 변의 길이 차이 허용 범위
        - angle_threshold : 변 간 평행성 판단을 위한 허용 각도 오차
    '''
    max_area = 0
    best_quad = None

    for quad in itertools.combinations(points, 4): # 가능한 4점 조합 생성
        valid, rect = is_valid_rectangle(quad, length_threshold, angle_threshold)
        if valid:
            area = polygon_area(rect) # 사각형 넓이 계산 
            if area > max_area: # 최대 넓이를 가지는 사각형 갱신
                max_area = area
                best_quad = rect

    return best_quad

# outlier에 맞게 threshold 조정한 함수(방식은 기존과 동일)
def detect_lines_outlier(edge_image, threshold=105, rho_threshold=30, theta_threshold=np.deg2rad(20)):
    height, width = edge_image.shape
    diag_len = int(np.ceil(np.sqrt(width**2 + height**2)))
    thetas = np.deg2rad(np.arange(-90, 90))
    rhos = np.arange(-diag_len, diag_len)
    accumulator = np.zeros((len(rhos), len(thetas)), dtype=np.uint64)

    y_idxs, x_idxs = np.nonzero(edge_image)

    for i in range(len(x_idxs)):
        x = x_idxs[i]
        y = y_idxs[i]
        for t_idx in range(len(thetas)):
            rho = int(round(x * np.cos(thetas[t_idx]) + y * np.sin(thetas[t_idx]))) + diag_len
            accumulator[rho, t_idx] += 1

    lines = []
    for r_idx in range(accumulator.shape[0]):
        for t_idx in range(accumulator.shape[1]):
            if accumulator[r_idx, t_idx] >= threshold:
                rho = rhos[r_idx]
                theta = thetas[t_idx]
                lines.append((rho, theta))
    # Remove similar lines
    filtered_lines = []
    for rho, theta in lines:
        is_similar = False
        for frho, ftheta in filtered_lines:
            if abs(rho - frho) < rho_threshold and abs(theta - ftheta) < theta_threshold:
                is_similar = True
                break
        if not is_similar:
            filtered_lines.append((rho, theta))

    return filtered_lines

def draw_lines(image, lines, padding=0):
    # detected lines 시각화 (패딩 보정 포함)
    for rho, theta in lines:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b)) + padding
        y1 = int(y0 + 1000 * (a)) + padding
        x2 = int(x0 - 1000 * (-b)) + padding
        y2 = int(y0 - 1000 * (a)) + padding
        cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    return image

def intersections_outlier(lines, image):
    # 기존 함수에서 outlier에 맞게 픽셀 거리 기준만 변경 
    height, width = image.shape[:2]
    intersections = []
    for (rho1, theta1), (rho2, theta2) in combinations(lines, 2):
        A = np.array([
            [np.cos(theta1), np.sin(theta1)],
            [np.cos(theta2), np.sin(theta2)]
        ])
        b = np.array([rho1, rho2])

        if np.linalg.det(A) != 0:
            x0, y0 = np.linalg.solve(A, b)
            x0 = int(round(x0))
            y0 = int(round(y0))
            if 0 <= x0 < width and 0 <= y0 < height:
                intersections.append((x0, y0))

    # 가까운 점 제거(근접한 점 병합)
    filtered_corners = []
    min_dist = 100  # 픽셀 거리 기준

    for pt in intersections:
        is_far = True
        for existing in filtered_corners:
            dist = np.linalg.norm(np.array(pt) - np.array(existing))
            if dist < min_dist:
                is_far = False
                break
        if is_far:
            filtered_corners.append(pt)

    return filtered_corners

def sort_corners_consistently(corners):
    # 임의 순서로 들어온 4개의 모서리 좌표(corners)를 일관된 순서(좌상, 우상, 우하, 좌하)로 정렬 
    pts = np.array(corners, dtype="float32")
    center = np.mean(pts, axis=0) # 중심점 계산 

    def angle_from_center(pt):
        return np.arctan2(pt[1] - center[1], pt[0] - center[0]) #  각 좌표에 대해 중심점 기준 각도(arctan2) 

    # 반시계 방향 정렬 
    pts_sorted = sorted(pts, key=angle_from_center)
    pts_sorted = np.array(pts_sorted, dtype="float32")

    s = pts_sorted.sum(axis=1)
    top_left_idx = np.argmin(s)
    pts_sorted = np.roll(pts_sorted, -top_left_idx, axis=0)

    # 정렬된 4개의 점인 np.array([top-left, top-right, bottom-right, bottom-left])를 반환
    return pts_sorted

def compute_auto_warp_size(pts):
    # 코너 자동 정렬을 위한 함수 
    # 네 개 코너를 각각 명시적으로 변수에 할당
    (tl, tr, br, bl) = pts

    # 너비: 상단과 하단 변의 길이 중 큰 것
    width_top = np.linalg.norm(tr - tl)
    width_bottom = np.linalg.norm(br - bl)
    width = max(int(width_top), int(width_bottom))

    # 높이: 좌측과 우측 변의 길이 중 큰 것
    height_left = np.linalg.norm(bl - tl)
    height_right = np.linalg.norm(br - tr)
    height = max(int(height_left), int(height_right))

    return width, height

def warp_perspective(img, corners):
    # 1. 코너 정렬
    src_pts = sort_corners_consistently(corners)

    # 2. 코너 기반으로 자동 크기 계산
    width_warp, height_warp = compute_auto_warp_size(src_pts)

    # 3. 목적지 좌표 정의
    dst_pts = np.array([
        [0, 0],
        [width_warp - 1, 0],
        [width_warp - 1, height_warp - 1],
        [0, height_warp - 1]
    ], dtype="float32")

    # 4. warping 행렬 계산 및 적용
    M = cv2.getPerspectiveTransform(src_pts, dst_pts)
    warped = cv2.warpPerspective(img, M, (width_warp, height_warp))

    return warped

`Executing Cell`
---

In [4]:
# 아래에 이미지 경로를 넣어 실행해주시면 됩니다.
img = cv2.imread(r"C:\Users\USER\Downloads\BC1.jpg")

# 이미지 전처리 -> edge 추출 -> 윤곽선 추출 
edge_preserved_img = preprocess_for_edge_detection(img)
closed_edges = detect_edges_and_close(edge_preserved_img)
quad_pts = get_largest_quad_contour(closed_edges, img.shape)

if quad_pts is not None:
    # 윤곽선이 추출된 경우, warping 실행행
    warped_card = warp_perspective(img, quad_pts)
    cv2.imshow("Warped Business Card", warped_card)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("1차 윤곽선 실패 → color_combined 사용")
    # 색상 기반 필터 + 기존 필터 
    quad_pts = color_combined(img, closed_edges)
    if quad_pts is not None:
        # 윤곽선이 추출된 경우 -> warping 실행 
        quad_pts1 = get_largest_quad_contour(quad_pts, img.shape)
        if quad_pts1 is not None:
            # warping 
            warped_card = warp_perspective(img, quad_pts1)
            print("검출 성공")
            cv2.imshow("Warped Business Card", warped_card)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        else:
            print("color_combined 검출 실패 line detect 실행")
            # line detect용 전처리 
            image = img
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            edge2 = cv2.Canny(gray, 20,70)
            edge3 = cv2.Canny(gray, 120,200)
            edge4 = edge2 - edge3

            # line detect 실행 
            lines = detect_lines(edge4)

            # corner 검출 
            filtered_corners = intersections(lines, image)
            corner_result = image.copy()
            for x, y in filtered_corners:
                cv2.circle(corner_result, (x, y), 7, (0, 255, 0), 2)

            if len(filtered_corners) == 4:
                # corner가 알맞게 검출된 경우 -> warping 실행 
                warped_card = warp_perspective(image, filtered_corners)
                cv2.imshow("Warped Business Card", warped_card)
                cv2.waitKey(0)
                cv2.destroyAllWindows()

            elif len(filtered_corners) < 4:
                # corner가 적은 경우(외곽에 corner 위치) 
                print("padding 실행")
                padding = 50 # padding할 픽셀 크기 
                image = cv2.copyMakeBorder(image, padding, padding, padding, padding, cv2.BORDER_CONSTANT, value=[255, 255, 255]) # 위, 아래, 좌, 우 모두 padding 
                # line detect와 동일한 전처리 
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                edge2 = cv2.Canny(gray, 20,70)
                edge3 = cv2.Canny(gray, 120,200)
                edge4 = edge2 - edge3

                # line detect 실행 
                lines = detect_lines(edge4, threshold=35, rho_threshold=80, theta_threshold=np.deg2rad(20))

                # corner 검출 
                filtered_corners = intersections(lines, image)
                corner_result = image.copy()
                for x, y in filtered_corners:
                    cv2.circle(corner_result, (x, y), 7, (0, 255, 0), 2)

                # warping 실행 
                warped_card = warp_perspective(image, filtered_corners)
                cv2.imshow("Warped Business Card", warped_card)
                cv2.waitKey(0)
                cv2.destroyAllWindows()

            else:
                if len(filtered_corners) > 10:
                    # BC 18, 20 -> 각각 특수한 코드로 실행 
                    # corner가 너무 많이 검출되는 경우 
                    print("Outlier")
                    # 윤곽선 기법에서의 전처리 실행 
                    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                    blurred = cv2.GaussianBlur(gray, (7, 7), 0)
                    bilateral = cv2.bilateralFilter(blurred, d=9, sigmaColor=75, sigmaSpace=75)
                    edge_preserved = cv2.edgePreservingFilter(bilateral, flags=1, sigma_s=60, sigma_r=0.4)
                    edges = cv2.Canny(edge_preserved, 75, 200)

                    # line detect 실행 
                    lines = detect_lines(edges)

                    # corner 검출 
                    filtered_corners = intersections(lines, image)
                    corner_result = image.copy()
                    for x, y in filtered_corners:
                        cv2.circle(corner_result, (x, y), 7, (0, 255, 0), 2)
                    if len(filtered_corners) == 4:
                        # BC 18
                        warped_card = warp_perspective(image, filtered_corners)
                        cv2.imshow("Warped Business Card", warped_card)
                        cv2.waitKey(0)
                        cv2.destroyAllWindows()

                    else:
                        # BC 20
                        # padding + 윤곽선 기법에서의 전처리 
                        padding = 50
                        image = cv2.copyMakeBorder(img, padding, padding, padding, padding, cv2.BORDER_CONSTANT, value=[255, 255, 255])
                        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                        blurred = cv2.GaussianBlur(gray, (7, 7), 0)
                        bilateral = cv2.bilateralFilter(blurred, d=9, sigmaColor=75, sigmaSpace=75)
                        edge_preserved = cv2.edgePreservingFilter(bilateral, flags=1, sigma_s=60, sigma_r=0.4)
                        edges = cv2.Canny(edge_preserved, 10, 20)

                        # line detect 실행 
                        lines = detect_lines_outlier(edges)
                        
                        # padding image 만들기 
                        original_image = image.copy()
                        draw_lines_image = draw_lines(original_image, lines, padding=padding)
                        
                        # corner 검출과 warping 
                        filtered_corners = intersections_outlier(lines, draw_lines_image)
                        warped_card = warp_perspective(img, filtered_corners)
                        cv2.imshow("Warped Business Card", warped_card)
                        cv2.waitKey(0)
                        cv2.destroyAllWindows()

                else:
                    # BC 13, 14
                    # corner가 기준인 4개보다 2~3개 정도 더 많은 경우 -> find largest rectangle 
                    print("Find largest rect")
                    corner4 = find_largest_rectangle(filtered_corners, length_threshold=100, angle_threshold=60)

                    # warping 실행 
                    warped_card = warp_perspective(image, corner4)
                    cv2.imshow("Warped Business Card", warped_card)
                    cv2.waitKey(0)
                    cv2.destroyAllWindows()
